In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
features = pd.read_csv('public_data_1/chems_feat.name',header=None).values.flatten()
labels = pd.read_csv('public_data_1/chems_label.name', header=None).values.flatten()
X = pd.read_csv('public_data_1/chems_train.data', header=None, names=features, sep=',')
y = pd.Series(pd.read_csv('public_data_1/chems_train.solution', header=None, names=labels, sep=',').values.flatten())

In [3]:
types = pd.read_csv('public_data_1/chems_feat.type', header=None).set_index(features)[0]
print(types.value_counts())

Numerical      152
Binary          12
Categorical      4
Name: 0, dtype: int64


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=1234)

In [5]:
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False, random_state=1234).fit(X_train, y_train)
print(roc_auc_score(y_test, lsvc.predict(X_test)))

0.8319862260270697


In [6]:
pd.Series(lsvc.coef_[0]).to_csv('submissions_aibek/selection.csv', index=False)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [8]:
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(C=0.01, penalty="l1", dual=False, random_state=1234))),
  ('classification', RandomForestClassifier(n_estimators=1000, random_state=1234))
])
clf.fit(X_train, y_train)
print(roc_auc_score(y_test, clf.predict(X_test)))

0.9052321966617247


In [9]:
test = pd.read_csv('public_data_1/chems_test.data', header=None, names=features, sep=',')
prediction = clf.predict(test)
pd.Series(prediction).to_csv('submissions_aibek/classification.csv', index=False)